In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# 시각화 스타일 설정
sns.set_theme(style="whitegrid", context="talk")
print("라이브러리 임포트 완료.")

In [ ]:
# 파일 경로 설정
file_path = Path("./validation_results.csv")

# 데이터 로드
try:
    df = pd.read_csv(file_path)
    print("데이터 로드 성공!")
    print(f"총 {len(df)}개의 데이터 포인트가 있습니다.")
except FileNotFoundError:
    print(f"오류: '{file_path}' 파일을 찾을 수 없습니다. 1_collect_validation_data.py를 먼저 실행하세요.")

# 데이터 타입 변환 및 기본적인 정보 확인
df['max_score'] = pd.to_numeric(df['max_score'], errors='coerce')
df['score_gap'] = pd.to_numeric(df['score_gap'], errors='coerce')
df['confidence_score'] = pd.to_numeric(df['confidence_score'], errors='coerce')

# 성공/실패를 시각화를 위해 1/0으로 변환
df['outcome_numeric'] = df['final_outcome'].apply(lambda x: 1 if x == 'success' else 0)

print("\n데이터 미리보기:")
df.head()

In [ ]:
# max_score와 score_gap에 따른 성공/실패 분포 시각화
plt.figure(figsize=(12, 8))

sns.scatterplot(
    data=df,
    x='max_score',
    y='score_gap',
    hue='final_outcome', # 성공/실패에 따라 색상 구분
    palette={'success': 'blue', 'failure': 'red'},
    alpha=0.7,
    s=100 # 점 크기
)

plt.title('메타인지 신호와 실제 성공/실패 분포', fontsize=20, pad=20)
plt.xlabel('최고 점수 (Max Score)', fontsize=15)
plt.ylabel('점수 격차 (Score Gap)', fontsize=15)
plt.legend(title='최종 결과', title_fontsize='13', fontsize='12')
plt.grid(True)
plt.show()

In [ ]:
# 규칙 후보 정의
path_a_rule = (df['max_score'] >= 8) & (df['score_gap'] >= 3)
path_c_rule = (df['max_score'] <= 4)

# 각 규칙에 해당하는 데이터 분리
df_path_a = df[path_a_rule]
df_path_c = df[path_c_rule]
df_path_b = df[~path_a_rule & ~path_c_rule] # A도 C도 아닌 나머지

# 각 경로의 통계 계산
path_stats = {
    'A (확신 실행)': {
        'Count': len(df_path_a),
        'Success Rate (%)': df_path_a['outcome_numeric'].mean() * 100 if len(df_path_a) > 0 else 0
    },
    'B (주의 실행)': {
        'Count': len(df_path_b),
        'Success Rate (%)': df_path_b['outcome_numeric'].mean() * 100 if len(df_path_b) > 0 else 0
    },
    'C (메타인지 합성)': {
        'Count': len(df_path_c),
        'Success Rate (%)': df_path_c['outcome_numeric'].mean() * 100 if len(df_path_c) > 0 else 0
    }
}

stats_df = pd.DataFrame(path_stats).T
print("정의된 규칙에 따른 각 경로의 통계:")
stats_df